# RESEAU DE NEURONE PROFOND POUR HISTORIQUE T-1

Le modèle a trois entrées: previous_action, previous_outcome, action.  Mais le previous outcome ne change pas la prediction.

# On crée le modèle

On initialise la seed pytorch CPU pour reproductibilité de l'entrainement.

In [1294]:
import torch

torch.manual_seed(42)

Modèle avec une couche cachée.

L'initialisation poids et bias des layers facilite beaucoup l'apprentissage !

In [1295]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(3, 6)
        # Apply He Initialization recommended for ReLU
        nn.init.kaiming_normal_(self.fc1.weight, mode='fan_in', nonlinearity='relu')
        
        self.fc2 = nn.Linear(6, 2)
        # Apply Xavier initialisation recommended for linear activation
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.zeros_(self.fc2.bias)  # Biases are usually set to zero

    def forward(self, x):
        x = torch.nn.functional.relu(self.fc1(x))  # Apply non-linearity
        return self.fc2(x)  # Logits (CrossEntropyLoss handles softmax)


In [1296]:
model = Model()

# On définit les paramètres d'apprentissage

In [1297]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()  # Cross-entropy for classification
optimizer = optim.SGD(model.parameters(), lr=0.3)  # SGD optimizer

# La fonction fit()

In [1298]:
def fit(inputs, targets):
    input_tensor = torch.tensor(inputs, dtype=torch.float)
    # input_tensor = torch.randn_like(input_tensor) * 0.01 (voir si le modèle apprend des tendances)
    target_tensor = torch.tensor(targets, dtype=torch.long)
    labels = torch.nn.functional.one_hot(target_tensor, num_classes=2).to(torch.float)
    #labels = torch.argmax(target_tensor, dim=1)  # Convert one-hot to class indices

    optimizer.zero_grad()  # Reset gradients
    outputs = model(input_tensor)  # Forward pass
    loss = criterion(outputs, labels)  # Compute loss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

    # Check accuracy (we expect 100% accuracy)
    predictions = torch.argmax(outputs, dim=1)
    accuracy = (predictions == target_tensor).float().mean().item()

    print(f"Loss: {loss.item():.6f}, Accuracy: {accuracy * 100:.0f}%")


# On entraine le modèle

In [1299]:
iteration = 1

In [1340]:
print("Iteration:", iteration)
fit([[2, 0, 2], [2, 0, 3], [3, 0, 2],[3, 0, 3], [2, 1, 2], [2, 1, 3], [3, 1, 2],[3, 1, 3]], [0, 1, 1, 0, 0, 1, 1, 0])
iteration += 1

Iteration: 41
Loss: 0.282971, Accuracy: 100%


On obtient souvent une accuracy de 100% apres environ 100 iterations.